In [7]:
from igraph import Graph
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import csv
import gseno
import os 
from matplotlib import rc
# Open the GraphML file and create a Graph object from it
mydir = os.getcwd()
g = Graph.Read_GraphML(mydir+"\\data\\networks\\weighted_graph_all_modes.GraphML")
Ncs = [1,2,3]
Csn_ncs = []
Anm_ncs = []

C:\Users\romul\AppData\Local\Temp\ipykernel_26320\1885318389.py:13: RuntimeWarning: Could not add vertex ids, there is already an 'id' vertex attribute. at src/io/graphml.c:492
  g = Graph.Read_GraphML(mydir+"\\data\\networks\\weighted_graph_all_modes.GraphML")


In [12]:
def filter_cases(csv_file, n):
    # Load the CSV file into a DataFrame
    df = pd.read_csv(
        csv_file,
        encoding='utf-8',
        sep=',',
        usecols=['ibgeID', 'newCases', 'totalCases', 'date'],
        dtype={'ibgeID': int}
    )

    # Convert 'date' column to datetime format
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

    # Filter the DataFrame based on the given conditions
    filtered_df = df[(df['totalCases'] >= n) & (df['newCases'] >= 1) & (df['ibgeID'] > 1000)]
    
    # Remove duplicate rows based on 'ibgeID'
    filtered_df = filtered_df.drop_duplicates(subset='ibgeID', keep='first')
    return filtered_df

def filter_records(df1, df2):
    """
    Returns a DataFrame containing the ibgeIDs present in both input DataFrames.
    """
    # Extract the ibgeIDs from both DataFrames
    ibgeIDs_df1 = set(df1['ibgeID'])
    ibgeIDs_df2 = set(df2['ibgeID'])

    # Find the intersection of both sets of ibgeIDs
    common_ibgeIDs = ibgeIDs_df1.intersection(ibgeIDs_df2)
    
    # Filter df2 to keep only rows with common ibgeIDs
    filtered_df2 = df2[df2['ibgeID'].isin(common_ibgeIDs)]
 
    # Sort the filtered DataFrame by 'date' and remove duplicate rows based on 'ibgeID' and 'date'
    filtered_df2 = (
        filtered_df2
        [['ibgeID', 'date']]
        .sort_values(by='date', ascending=True)
        .drop_duplicates(subset=['ibgeID', 'date'])
    )

    # Return the filtered DataFrame
    return filtered_df2

In [13]:


for nc in Ncs:  
    # Set vertex labels
    g.vs['label'] = np.linspace(1, g.vcount(), g.vcount(), dtype=int).tolist()

    # Inverse weights for betweenness calculation
    g.es['w_inv'] = 1.0 / np.array(g.es['weight'])

    # Calculate various graph metrics
    g.vs['betweenness'] = g.betweenness(vertices=None, directed=False, cutoff=None, weights='w_inv')
    g.vs['clustering'] = g.transitivity_local_undirected()
    g.vs['strength'] = g.strength(weights="weight")
    g.vs['closeness'] = g.closeness(vertices=None, mode='all', cutoff=None, weights='w_inv', normalized=True)
    g.vs['eigenvector'] = g.evcent(directed=False, scale=True, weights='w_inv', return_eigenvalue=False)

    # Create a DataFrame with graph metrics
    graph_df = pd.DataFrame({
        'label': g.vs['label'],
        'ibgeID': g.vs["geocode"],
        'betweenness': g.vs['betweenness'],
        'clustering': g.vs['clustering'],
        'strength': g.vs['strength'],
        'closeness': g.vs['closeness'],
        'eigenvector': g.vs['eigenvector']
    })

    # Filter the cases DataFrame
    filter_df = filter_cases(mydir+"\\data\\preprocessed\\cases-brazil-cities-time_2020.csv", nc)
    # Get the shared ibgeIDs between the graph DataFrame and filtered cases DataFrame
    shared_cities = filter_records(graph_df, filter_df)
    shared_cities = shared_cities.rename(columns={'ibgeID': 'label', 'date': 'rank'})
    shared_cities['label'] = shared_cities['label'].astype(float)

    shared_cities_set = set(shared_cities['label'])

    # Filter vertices to keep only those in shared_cities
    vertices_to_keep = []
    for v in g.vs:
        geocode = v['geocode']
        if geocode in shared_cities_set:
            vertices_to_keep.append(v.index)
        else:
            print(f"Vertex {v.index} with geocode {geocode} is not in shared_cities")

    g.vs['label'] = g.vs['geocode']
    subgraph = g.subgraph(vertices_to_keep)

    # First approach: Compute the intersections between sets of nodes ordered by a certain metric and its rank from the input DataFrame
    results_df = gseno.csn(
        graph=subgraph, 
        sorted_nodes_df=shared_cities   
    )
    Csn_ncs.append(results_df)


    # Second approach: Accumulate the nodes' metrics from the sorted nodes by rank (from the input DataFrame)
    # The output for each metric is normalized to be within [0,1]
    results_df = gseno.anm(
        graph=subgraph, 
        sorted_nodes_df=shared_cities
    )
    Anm_ncs.append(results_df)


    #setUp_comparsion_table(min_cases[0])
    #print(degrees_similarity)

Vertex 2903 with geocode 3165206.0 is not in shared_cities


In [15]:
print("Csn_ncs")
print(Csn_ncs)

Csn_ncs
[             degrees  betweenness  clustering  strength  closeness_w   eignv_w
2020-02-25  1.000000     0.000000    0.000000  1.000000     0.000000  1.000000
2020-03-05  0.500000     0.000000    0.000000  0.500000     0.000000  0.500000
2020-03-06  0.250000     0.250000    0.000000  0.250000     0.000000  0.250000
2020-03-07  0.500000     0.166667    0.000000  0.333333     0.000000  0.333333
2020-03-08  0.500000     0.125000    0.000000  0.250000     0.000000  0.250000
...              ...          ...         ...       ...          ...       ...
2020-10-12  0.999257     0.999257    0.999257  0.999257     0.999257  0.999257
2020-10-20  0.999443     0.999443    0.999443  0.999443     0.999443  0.999443
2020-10-27  0.999628     0.999628    0.999628  0.999628     0.999628  0.999628
2020-11-06  0.999814     0.999814    0.999814  0.999814     0.999814  0.999814
2020-11-13  1.000000     1.000000    1.000000  1.000000     1.000000  1.000000

[194 rows x 6 columns],              degre

In [16]:
print("Anm_ncs")
print(Anm_ncs)

Anm_ncs
[             degrees  betweenness  clustering  strength  closeness_w   eignv_w
2020-02-25  0.011154     0.024371    0.000007  0.011318     0.000286  0.007686
2020-03-05  0.011932     0.024628    0.000066  0.011855     0.000517  0.008309
2020-03-06  0.018780     0.037561    0.000113  0.019186     0.001069  0.013860
2020-03-07  0.023154     0.050880    0.000135  0.024489     0.001429  0.018231
2020-03-08  0.025597     0.056453    0.000240  0.027119     0.001950  0.020234
...              ...          ...         ...       ...          ...       ...
2020-10-12  0.999908     1.000000    0.999246  0.999877     0.999205  0.999975
2020-10-20  0.999916     1.000000    0.999246  0.999920     0.999450  0.999975
2020-10-27  0.999939     1.000000    0.999536  0.999946     0.999665  0.999994
2020-11-06  0.999985     1.000000    0.999710  0.999989     0.999834  0.999999
2020-11-13  1.000000     1.000000    1.000000  1.000000     1.000000  1.000000

[194 rows x 6 columns],              degre